## ei_runner_mp.ipynb 
A version of ei_runner capable of multiprocess simulations using the ei_net class.  
By Sergio Verduzco  
January 2018

In [1]:
# importing libraries
%cd ..
from ei_net import *
from pathos.multiprocessing import ProcessingPool
import sys 

/home/z/projects/draculab


In [2]:
# This is the name of the file where logging will happen
log_file = "./logs/ei_net_mp" + time.strftime("_%m-%d-%y.txt")

In [ ]:
%%capture capt 
# logging draculab version used for this session
print('#======================================================')
print('#             | Latest git commit: |')
print('#====================================================== \n#', end=' ')
!git log -1 --oneline --color=never

In [ ]:
# Write the output of the cell above in the simulation log
f = open(log_file, mode='a')
f.write(capt.stdout)
f.close()

In [3]:
# Creating input patterns
n_pat = 4  # number of input patterns
pat_arr = np.zeros((4, 25)) # ei_runner.n['x'])) # array with all input patterns

for row in range(5):
    for col in range(5):
        idx = 5*col + row
        if row == col or row+col == 4: # The X input pattern
            pat_arr[0,idx] = 1
        if row == 2 or col == 2:  # The + input pattern
            pat_arr[1,idx] = 1
        if row == 1 or row == 3:  # The = input pattern
            pat_arr[2,idx] = 1
        if col == 1 or col == 3:  # The || input pattern
            pat_arr[3,idx] = 1
pat_arr = pat_arr / 5.

def inp_pat(pres, rows, columns, random=False):
    # The function that creates the input pattern shown at each presentation in ei_net.run
    if random:
        idx = np.random.randint(4)
    else:
        idx = pres % 4
    return pat_arr[idx, :]
#for i in range(6):
#    print(np.reshape(inp_pat(5, 4, 2),(5,5)))

In [4]:
class mp_ei_runner():
    def __init__(self):
        self.name = 'mp_ei_runner_test_nprocs'
        self.n_procs = 14
        self.n_nets = 14
        self.n_pres = [8]*self.n_nets  # number of presentations for each net
        self.pres_t = [1.]*self.n_nets # duration of presentations for each net
        self.inp_pats = [inp_pat]*self.n_nets # input pattern creator for each net
        self.inp_funcs = [None]*self.n_nets  # input function creator for each net. None sets the default.
        
        
        #F = open("/home/z/projects/draculab/pickles/mp_ei_runner_test0.pkl", 'rb')
        #dill_obj = dill.load(F)
        #self.nets = dill_obj.nets
        
        self.nets = [ei_net(i) for i in range(self.n_nets)] # First run
        
        # Setting the parameter changes for all networks
        ## First, set baseline changes
        for idx, net in enumerate(self.nets):
            net.annotate('This is network ' + str(idx) + ' in a run of an mp_ei_runner object called ' + self.name)
            net.annotate('These tests are mainly about exp_dist_sig units vs exp_dist_sig_thr.')
            #----------------------------------
            net.annotate("Adjusting size of e,i groups.")
            net.set_param('e_geom', 'rows', 16)
            net.set_param('e_geom', 'columns', 16)
            net.set_param('i_geom', 'rows', 8)
            net.set_param('i_geom', 'columns', 8)
            net.annotate("Compensating for the ratio of excitatory to inhibitory units.")
            net.set_param('ee_conn', 'weights',{'uniform' : {'low': .01, 'high' : 0.3}})
            net.set_param('ei_conn', 'weights',{'uniform' : {'low': .01, 'high' : 0.3}})
            net.set_param('ie_conn', 'weights',{'uniform' : {'low': -1.3, 'high' : -0.01}})
            net.set_param('ii_conn', 'weights',{'uniform' : {'low': -1., 'high' : -0.01}})
            #----------------------------------
            net.annotate("Setting input parameters")
            net.set_param('x_geom', 'rows', 5) 
            net.set_param('x_geom', 'columns', 5) 
            net.set_param('xe_conn', 'weights', {'uniform' : {'low': 0.05, 'high' : 4.}})
            net.set_param('xi_conn', 'weights', {'uniform' : {'low': 0.05, 'high' : 4.}})
            net.set_param('xe_conn', 'mask', {'circular': {'radius': .3}})
            net.set_param('xi_conn', 'mask', {'circular': {'radius': .3}})
            #----------------------------------
            net.annotate("Using Oja input synapses")
            net.set_param('xe_syn', 'type', synapse_types.oja)
            net.set_param('xe_syn', 'lrate', 1./100.)
            net.set_param('xi_syn', 'type', synapse_types.oja)
            net.set_param('xi_syn', 'lrate', 1./100.)
            #----------------------------------
            net.annotate("Using sq_hebb_subsnorm EE synapses")
            net.set_param('ee_syn', 'type', synapse_types.sq_hebbsnorm)
            net.set_param('ee_syn', 'lrate', 1./100.)
            net.set_param('ee_syn', 'omega', 2.)
            #----------------------------------
            net.annotate("Using corr_homeo_inhib IE synapses")
            net.set_param('ie_syn', 'type', synapse_types.corr_inh)
            net.set_param('ie_syn', 'lrate', 1./50.)
            net.set_param('e_pars', 'des_act', 0.4)
            #----------------------------------
            net.annotate("Using anti_cov_pre EI synapses")
            net.set_param('ei_syn', 'type', synapse_types.anticov_pre)
            net.set_param('ei_syn', 'lrate', .01)
            #----------------------------------
            net.annotate("Using exp_dist_sig_thr units for the excitatory population.")
            net.set_param('e_pars', 'type', unit_types.exp_dist_sig_thr)
            net.set_param('e_pars', 'tau_thr', 0.005)
            net.set_param('e_pars', 'c', 3.)
            #----------------------------------
            net.set_param('n', 'w_track', 16)
            #----------------------------------
        ## Now adjust networks individually
        ### network 0 is the baseline
        ### network 1
        self.nets[1].annotate("Using exp_dist_sig units for the excitatory population.")
        self.nets[1].set_param('e_pars', 'type', unit_types.exp_dist_sig)
        self.nets[1].set_param('e_pars', 'tau_scale', 0.05)
        self.nets[1].set_param('e_pars', 'c', 3.)
        self.nets[1].set_param('e_pars', 'Kp', .05)
        ### network 2
        self.nets[2].annotate("Using exp_dist_sig units for the excitatory population, and fast IE plasticity.")
        self.nets[2].set_param('e_pars', 'type', unit_types.exp_dist_sig)
        self.nets[2].set_param('e_pars', 'tau_scale', 0.05)
        self.nets[2].set_param('e_pars', 'c', 3.)
        self.nets[2].set_param('e_pars', 'Kp', .05)
        self.nets[2].set_param('ei_syn', 'lrate', .1)
        ### network 3
        self.nets[3].annotate('This network has exp_dist_sig exc units, and 2X faster learning rates, 0.7s presentations. ')
        self.nets[3].set_param('xe_syn', 'lrate', 1./50.)
        self.nets[3].set_param('xi_syn', 'lrate', 1./50.)
        self.nets[3].set_param('ee_syn', 'lrate', 1./50.)
        self.nets[3].set_param('ie_syn', 'lrate', 1./25.)
        self.nets[3].set_param('ei_syn', 'lrate', .02)
        self.nets[3].set_param('e_pars', 'tau_thr', 0.005)
        self.nets[3].set_param('e_pars', 'type', unit_types.exp_dist_sig)
        self.nets[3].set_param('e_pars', 'tau_scale', 0.05)
        self.nets[3].set_param('e_pars', 'c', 3.)
        self.nets[3].set_param('e_pars', 'Kp', .05)
        self.nets[3].set_param('ei_syn', 'lrate', .05)
        self.pres_t[3] = 0.7
        ### Network 4
        self.nets[4].annotate("Using exp_dist_sig units for the excitatory population, fast IE plasticity, and exponent 1 for the rate distro.")
        self.nets[4].set_param('e_pars', 'type', unit_types.exp_dist_sig)
        self.nets[4].set_param('e_pars', 'tau_scale', 0.05)
        self.nets[4].set_param('e_pars', 'c', 1.)
        self.nets[4].set_param('e_pars', 'Kp', .05)
        self.nets[4].set_param('ei_syn', 'lrate', .1)
        ### Network 5
        self.nets[5].annotate('This network has exp_dist_sig exc units, and covariance II synapses')
        self.nets[5].set_param('ii_syn', 'type', synapse_types.cov)
        self.nets[5].set_param('ii_syn', 'lrate', 0.01)
        self.nets[5].set_param('e_pars', 'type', unit_types.exp_dist_sig)
        self.nets[5].set_param('e_pars', 'tau_scale', 0.05)
        self.nets[5].set_param('e_pars', 'c', 3.)
        self.nets[5].set_param('e_pars', 'Kp', .05)
        self.nets[5].set_param('ei_syn', 'lrate', .1)
        ### Network 6
        self.nets[6].annotate('This network has exp_dist_sig_thr units, and anti_covariance II synapses')
        self.nets[6].set_param('ii_syn', 'type', synapse_types.anticov)
        self.nets[6].set_param('ii_syn', 'lrate', 0.01)
        
        # Building the networks
        for net in self.nets:
            net.build()
        
        
    def run_net(self, net_num):
        print('Running network %d' % (net_num))
        return self.nets[net_num].run(self.n_pres[net_num], self.pres_t[net_num], self.inp_pats[net_num], self.inp_funcs[net_num])
    
    
    def run_all(self, save=True, log=False):
        sys.setrecursionlimit(5000)  # otherwise the networks can't be pickled
        pool = ProcessingPool(nodes=self.n_procs) 
        print('Starting %d processes, %d networks' % (self.n_procs, self.n_nets))
        start_time = time.time()
        self.nets = pool.map(self.run_net, range(self.n_nets))
        print('****** Processing finished after %s seconds ******' % (time.time() - start_time)) 
        pool.close()
        pool.join()
        if save:
            for net in self.nets:
                net.history.append('ei_net object being saved in container mp_ei_runner object ' + self.name)
            F = open(self.name+'.pkl', 'wb')
            dill.dump(self, F)
            F.close()
        if log:
            self.log()
                
    def log(self, file_name=log_file):
        for net in self.nets:
            net.log(file_name, params=(net.net_number == 0))

In [5]:
# Simulating
mp_ei_obj = mp_ei_runner()
mp_ei_obj.run_all(save=False, log=False)

Starting 14 processes, 14 networks
Running network 0
Starting presentation 0 at network 0
Running network 1
Starting presentation 0 at network 1
Presentation 0 took 61.64643931388855 seconds at network 0.
Starting presentation 1 at network 0
Running network 2
Starting presentation 0 at network 2
Running network 3
Starting presentation 0 at network 3
Presentation 0 took 109.03572535514832 seconds at network 1.
Starting presentation 1 at network 1
Presentation 1 took 83.99352097511292 seconds at network 0.
Starting presentation 2 at network 0
Presentation 0 took 92.80180668830872 seconds at network 2.
Starting presentation 1 at network 2
Running network 4
Starting presentation 0 at network 4
Presentation 0 took 53.3207311630249 seconds at network 3.
Starting presentation 1 at network 3
Running network 5
Starting presentation 0 at network 5
Presentation 2 took 62.80190563201904 seconds at network 0.
Starting presentation 3 at network 0
Presentation 1 took 84.54903793334961 seconds at netw

Starting presentation 6 at network 4
Presentation 2 took 79.42941546440125 seconds at network 12.
Starting presentation 3 at network 12
Presentation 4 took 75.91346955299377 seconds at network 10.
Starting presentation 5 at network 10
Presentation 4 took 74.69703555107117 seconds at network 9.
Starting presentation 5 at network 9
Presentation 2 took 61.470988035202026 seconds at network 13.
Starting presentation 3 at network 13
Presentation 7 took 65.0742700099945 seconds at network 6.
Total execution time is 546.9679684638977 seconds at network 6
----------------------
Presentation 5 took 75.04394721984863 seconds at network 7.
Starting presentation 6 at network 7
Presentation 6 took 82.51906752586365 seconds at network 5.
Starting presentation 7 at network 5
Presentation 3 took 63.396990060806274 seconds at network 11.
Starting presentation 4 at network 11
Presentation 7 took 91.67082571983337 seconds at network 1.
Total execution time is 779.5372767448425 seconds at network 1
------

In [ ]:
# basic plot 
%matplotlib inline
net_num = 6
ei_runner = mp_ei_obj.nets[net_num]
print(ei_runner.notes)
ei_runner.basic_plot()

In [ ]:
ei_runner.double_anim(ei_runner.e, slider=False, interv=20, nbins=20, thr=0.9, pdf=True)

In [ ]:
# Plot the highest excitatory unit activities
%matplotlib inline
n_top = 5  # how many units to plot
top_pop = np.array(ei_runner.e)  # population to visualize
top_units = ei_runner.all_activs[top_pop].sum(1).argpartition(-n_top)[-n_top:]
print(top_units)
top_act_fig = plt.figure(figsize=(18,8))
top_acts = ei_runner.all_activs[top_pop[top_units]].transpose()
plt.plot(ei_runner.all_times, top_acts)
plt.show()

In [ ]:
# Plot the lowest excitatory unit activities
n_bottom = 8  # how many units to plot plus one
bottom_pop = np.array(ei_runner.e)  # population to visualize
bottom_units = ei_runner.all_activs[bottom_pop].sum(1).argpartition(n_bottom)[0:n_bottom-1]
print(bottom_units)
bottom_act_fig = plt.figure(figsize=(18,8))
bottom_acts = ei_runner.all_activs[bottom_pop[bottom_units]].transpose()
plt.plot(ei_runner.all_times, bottom_acts)
plt.show()

In [ ]:
# Annotations and log
ei_runner.annotate('These simulations show that even when the exponent of the distribution is negative, ', make_history=False)
ei_runner.annotate('exp_dist_sig_thr units can reasonably maintain the distribution and make the network', make_history=False)
ei_runner.annotate('selective, unlike exp_dist_sig units.', make_history=False)
#ei_runner.annotate('2) the response to each pattern consists of a largely non-overlaping set of units,', make_history=False)
#ei_runner.annotate('3) the steady-state response roughly follows the exponential distribution.', make_history=False)
ei_runner.log(name=log_file,params=False)

In [ ]:
ei_runner.history

In [ ]:
ei_runner.conn_anim(ei_runner.x, ei_runner.e, slider=True, weights=True)

In [ ]:
ei_runner.act_anim(ei_runner.e, 0.9, interv=20, slider=True)

In [ ]:
ei_runner.hist_anim(ei_runner.e, slider=True, nbins=20, pdf=True)

In [ ]:
F = open("/home/z/projects/draculab/pickles/mp_ei_runner_test2.pkl", 'rb')
mp_ei_obj = dill.load(F)

In [ ]:
# Simulating a single network
n_pres = 8
pres_time = 1.
ei_runner.run(n_pres, pres_time, set_inp_pat=inp_pat)

In [11]:
class dog:
    def __init__(self,parts):
        for part in parts:
            setattr(self, part, 'chunk')
            

In [12]:
fido = dog(['paws', 'tail', 'snot'])
dir(fido)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'paws',
 'snot',
 'tail']

In [14]:
fido.paws

'chunk'